# 7차시: 배포 및 통합

## 학습 목표
- FastAPI 기반 REST API 서버 구축
- Docker 컨테이너화 및 배포 환경 구성
- CI/CD 파이프라인 구축
- 모니터링 및 운영 도구 구성
- 프로덕션 환경 배포 전략

## 목차
1. [환경 설정](#환경-설정)
2. [FastAPI 서버 구축](#fastapi-서버-구축)
3. [Docker 컨테이너화](#docker-컨테이너화)
4. [CI/CD 파이프라인](#cicd-파이프라인)
5. [모니터링 설정](#모니터링-설정)
6. [배포 테스트](#배포-테스트)

## 환경 설정

프로덕션 배포에 필요한 라이브러리를 설치하고 설정합니다.

In [ ]:
# 필요한 라이브러리 설치 (주석 해제하여 실행)
# !pip install fastapi uvicorn docker prometheus-client structlog opentelemetry-api

print("배포 관련 라이브러리 설치 완료!")

In [ ]:
#!/usr/bin/env python3
"""
7차시: 배포 및 통합
Author: AI Chatbot Workshop
Date: 2024-08-30
Description: FastAPI REST API, Docker 배포, CI/CD 파이프라인, 프로덕션 환경 구성
"""

import os
import sys
import json
import time
import uuid
import logging
import asyncio
from typing import Dict, List, Any, Optional
from dataclasses import dataclass, asdict
from datetime import datetime, timedelta
from pathlib import Path
import subprocess

# FastAPI 및 웹 서버
from fastapi import FastAPI, HTTPException, Depends, BackgroundTasks, Request, Response
from fastapi.security import HTTPBearer, HTTPAuthorizationCredentials
from fastapi.middleware.cors import CORSMiddleware
from fastapi.responses import JSONResponse, StreamingResponse
from pydantic import BaseModel, Field
import uvicorn

# 모니터링 및 로깅
from prometheus_client import Counter, Histogram, Gauge, generate_latest, CONTENT_TYPE_LATEST
import structlog

# Docker
import docker

# 로컬 모듈
sys.path.append('..')
from config import get_config

# 설정
config = get_config()
logger = structlog.get_logger(__name__)

print("배포 관련 모듈 임포트 완료!")

## 1. FastAPI 서버 구축

### 1.1 기본 API 모델 정의
API 요청/응답을 위한 데이터 모델을 정의합니다.

In [ ]:
# Prometheus 메트릭 정의
API_REQUESTS_TOTAL = Counter('api_requests_total', 'Total API requests', ['method', 'endpoint', 'status'])
API_REQUEST_DURATION = Histogram('api_request_duration_seconds', 'API request duration')
ACTIVE_CONNECTIONS = Gauge('api_active_connections', 'Active API connections')

# API 데이터 모델
class ChatRequest(BaseModel):
    """채팅 요청 모델"""
    message: str = Field(..., description="사용자 메시지")
    conversation_id: Optional[str] = Field(None, description="대화 ID")
    model: str = Field("gpt-3.5-turbo", description="사용할 모델")
    temperature: float = Field(0.7, description="응답 창의성")
    max_tokens: Optional[int] = Field(None, description="최대 토큰 수")
    stream: bool = Field(False, description="스트리밍 응답 여부")
    use_cache: bool = Field(True, description="캐시 사용 여부")

class ChatResponse(BaseModel):
    """채팅 응답 모델"""
    response: str
    conversation_id: str
    model: str
    usage: Dict[str, int]
    response_time: float
    cached: bool
    timestamp: str

class HealthResponse(BaseModel):
    """헬스 체크 응답"""
    status: str
    timestamp: str
    version: str
    uptime: float
    services: Dict[str, str]

# 보안
security = HTTPBearer(auto_error=False)

print("API 모델 정의 완료!")

### 1.2 FastAPI 애플리케이션 구성
미들웨어와 라우터를 포함한 완전한 API 서버를 구성합니다.

In [ ]:
class ProductionChatbotAPI:
    """프로덕션용 챗봇 API 서버"""
    
    def __init__(self):
        """API 서버 초기화"""
        self.app = FastAPI(
            title="AI Chatbot API",
            description="Production-ready AI Chatbot API",
            version="1.0.0",
            docs_url="/docs",
            redoc_url="/redoc"
        )
        
        # 시작 시간
        self.start_time = time.time()
        
        # 미들웨어 설정
        self._setup_middleware()
        
        # 라우터 설정
        self._setup_routes()
    
    def _setup_middleware(self):
        """미들웨어 설정"""
        # CORS 설정
        self.app.add_middleware(
            CORSMiddleware,
            allow_origins=["http://localhost:3000", "http://localhost:8080"],
            allow_credentials=True,
            allow_methods=["GET", "POST"],
            allow_headers=["*"],
        )
        
        # 커스텀 모니터링 미들웨어
        @self.app.middleware("http")
        async def monitoring_middleware(request: Request, call_next):
            """요청 모니터링 미들웨어"""
            start_time = time.time()
            
            # 활성 연결 수 증가
            ACTIVE_CONNECTIONS.inc()
            
            try:
                response = await call_next(request)
                
                # 메트릭 기록
                duration = time.time() - start_time
                API_REQUEST_DURATION.observe(duration)
                API_REQUESTS_TOTAL.labels(
                    method=request.method,
                    endpoint=request.url.path,
                    status=response.status_code
                ).inc()
                
                # 응답 헤더에 추가 정보
                response.headers["X-Response-Time"] = str(duration)
                response.headers["X-Request-ID"] = str(uuid.uuid4())
                
                return response
                
            except Exception as e:
                # 에러 메트릭
                API_REQUESTS_TOTAL.labels(
                    method=request.method,
                    endpoint=request.url.path,
                    status=500
                ).inc()
                
                logger.error(f"요청 처리 중 오류: {e}")
                raise
            
            finally:
                # 활성 연결 수 감소
                ACTIVE_CONNECTIONS.dec()
    
    def _setup_routes(self):
        """API 라우터 설정"""
        
        @self.app.get("/health", response_model=HealthResponse)
        async def health_check():
            """헬스 체크"""
            uptime = time.time() - self.start_time
            
            # 서비스 상태 확인
            services = {
                "api": "healthy",
                "database": "healthy",  # 실제 구현에서는 DB 연결 확인
                "redis": "healthy",     # 실제 구현에서는 Redis 연결 확인
                "openai": "healthy"     # 실제 구현에서는 API 키 검증
            }
            
            status = "healthy" if all(s == "healthy" for s in services.values()) else "degraded"
            
            return HealthResponse(
                status=status,
                timestamp=datetime.now().isoformat(),
                version="1.0.0",
                uptime=uptime,
                services=services
            )
        
        @self.app.get("/metrics")
        async def metrics():
            """Prometheus 메트릭 엔드포인트"""
            return Response(generate_latest(), media_type=CONTENT_TYPE_LATEST)
        
        @self.app.post("/chat", response_model=ChatResponse)
        async def chat_completion(
            request: ChatRequest,
            background_tasks: BackgroundTasks,
            auth: Optional[HTTPAuthorizationCredentials] = Depends(security)
        ):
            """채팅 완성 API"""
            start_time = time.time()
            
            try:
                # 대화 ID 생성
                conversation_id = request.conversation_id or str(uuid.uuid4())
                
                # 시뮬레이션된 응답 (실제 구현에서는 이전 차시 코드 활용)
                await asyncio.sleep(0.1)  # API 호출 시뮬레이션
                
                response_text = f"안녕하세요! '{request.message}'에 대한 답변입니다. 이것은 테스트 응답입니다."
                
                # 응답 구성
                response = ChatResponse(
                    response=response_text,
                    conversation_id=conversation_id,
                    model=request.model,
                    usage={
                        "prompt_tokens": len(request.message.split()) * 2,
                        "completion_tokens": len(response_text.split()) * 2,
                        "total_tokens": (len(request.message) + len(response_text)) * 2
                    },
                    response_time=time.time() - start_time,
                    cached=False,
                    timestamp=datetime.now().isoformat()
                )
                
                return response
                
            except Exception as e:
                logger.error(f"채팅 완성 오류: {e}")
                raise HTTPException(status_code=500, detail=f"내부 서버 오류: {str(e)}")
        
        @self.app.get("/")
        async def root():
            """루트 엔드포인트"""
            return {
                "message": "AI Chatbot API",
                "version": "1.0.0",
                "docs_url": "/docs",
                "health_url": "/health"
            }

# API 서버 인스턴스 생성
api_server = ProductionChatbotAPI()
app = api_server.app

print("FastAPI 애플리케이션 구성 완료!")
print("다음 명령어로 서버를 실행할 수 있습니다:")
print("uvicorn lesson7_deployment_integration:app --host 0.0.0.0 --port 8000 --reload")

## 2. Docker 컨테이너화

### 2.1 Docker 파일 생성
애플리케이션을 컨테이너화하기 위한 Dockerfile을 생성합니다.

In [ ]:
class DockerManager:
    """Docker 배포 관리자"""
    
    def __init__(self):
        """Docker 매니저 초기화"""
        try:
            self.client = docker.from_env()
            self.logger = structlog.get_logger(__name__)
            self.logger.info("Docker 클라이언트 초기화 완료")
        except Exception as e:
            print(f"Docker 클라이언트 초기화 실패: {e}")
            print("Docker가 설치되어 있고 실행 중인지 확인하세요.")
            self.client = None
            self.logger = None
    
    def generate_dockerfile(self, output_path: str = "Dockerfile"):
        """Dockerfile 생성"""
        dockerfile_content = '''# 멀티스테이지 빌드
FROM python:3.9-slim as builder

# 시스템 의존성 설치
RUN apt-get update && apt-get install -y \\
    build-essential \\
    curl \\
    && rm -rf /var/lib/apt/lists/*

# Python 의존성 설치
COPY requirements.txt .
RUN pip install --no-cache-dir --user -r requirements.txt

# 프로덕션 스테이지
FROM python:3.9-slim

# 시스템 사용자 생성
RUN groupadd -r appuser && useradd -r -g appuser appuser

# 필요한 시스템 패키지 설치
RUN apt-get update && apt-get install -y \\
    curl \\
    && rm -rf /var/lib/apt/lists/*

# Python 패키지 복사
COPY --from=builder /root/.local /home/appuser/.local

# 애플리케이션 디렉토리 생성
WORKDIR /app

# 소스 코드 복사
COPY --chown=appuser:appuser . .

# 포트 노출
EXPOSE 8000

# 헬스 체크
HEALTHCHECK --interval=30s --timeout=30s --start-period=5s --retries=3 \\
    CMD curl -f http://localhost:8000/health || exit 1

# 사용자 변경
USER appuser

# 환경 변수
ENV PATH=/home/appuser/.local/bin:$PATH
ENV PYTHONPATH=/app

# 애플리케이션 실행
CMD ["uvicorn", "lesson7.lesson7_deployment_integration:app", "--host", "0.0.0.0", "--port", "8000"]
'''
        
        with open(output_path, 'w') as f:
            f.write(dockerfile_content)
        
        print(f"Dockerfile 생성 완료: {output_path}")
        return output_path
    
    def generate_docker_compose(self, output_path: str = "docker-compose.yml"):
        """docker-compose.yml 생성"""
        compose_content = '''version: '3.8'

services:
  chatbot-api:
    build:
      context: .
      dockerfile: Dockerfile
    container_name: chatbot-api
    ports:
      - "8000:8000"
    environment:
      - REDIS_HOST=redis
      - REDIS_PORT=6379
      - ENVIRONMENT=production
    volumes:
      - ./logs:/app/logs
      - ./data:/app/data
    depends_on:
      - redis
      - prometheus
    restart: unless-stopped
    networks:
      - chatbot-network

  redis:
    image: redis:7-alpine
    container_name: chatbot-redis
    ports:
      - "6379:6379"
    volumes:
      - redis_data:/data
    restart: unless-stopped
    networks:
      - chatbot-network

  prometheus:
    image: prom/prometheus:latest
    container_name: chatbot-prometheus
    ports:
      - "9090:9090"
    volumes:
      - ./monitoring/prometheus.yml:/etc/prometheus/prometheus.yml
      - prometheus_data:/prometheus
    restart: unless-stopped
    networks:
      - chatbot-network

  grafana:
    image: grafana/grafana:latest
    container_name: chatbot-grafana
    ports:
      - "3000:3000"
    volumes:
      - grafana_data:/var/lib/grafana
    environment:
      - GF_SECURITY_ADMIN_PASSWORD=admin
    restart: unless-stopped
    networks:
      - chatbot-network

volumes:
  redis_data:
  prometheus_data:
  grafana_data:

networks:
  chatbot-network:
    driver: bridge
'''
        
        with open(output_path, 'w') as f:
            f.write(compose_content)
        
        print(f"docker-compose.yml 생성 완료: {output_path}")
        return output_path
    
    def create_deployment_script(self, output_path: str = "deploy.sh"):
        """배포 스크립트 생성"""
        deploy_script = '''#!/bin/bash
set -e

echo "=== AI 챗봇 API 배포 시작 ==="

# 환경 변수 확인
if [ -z "$OPENAI_API_KEY" ]; then
    echo "OPENAI_API_KEY 환경 변수가 설정되지 않았습니다."
    exit 1
fi

# Docker 이미지 빌드
echo "Docker 이미지 빌드 중..."
docker-compose build --no-cache

# 기존 컨테이너 중지
echo "기존 서비스 중지 중..."
docker-compose down

# 새 컨테이너 시작
echo "새 서비스 시작 중..."
docker-compose up -d

# 헬스 체크
echo "헬스 체크 중..."
sleep 30
curl -f http://localhost:8000/health || exit 1

echo "=== 배포 완료 ==="
echo "API 문서: http://localhost:8000/docs"
echo "Grafana: http://localhost:3000 (admin/admin)"
echo "Prometheus: http://localhost:9090"
'''
        
        with open(output_path, 'w') as f:
            f.write(deploy_script)
        
        # 실행 권한 부여
        os.chmod(output_path, 0o755)
        
        print(f"배포 스크립트 생성 완료: {output_path}")
        return output_path

# Docker 관리자 테스트
docker_manager = DockerManager()

if docker_manager.client:
    # Docker 파일들 생성
    dockerfile_path = docker_manager.generate_dockerfile()
    compose_path = docker_manager.generate_docker_compose()
    deploy_script_path = docker_manager.create_deployment_script()
    
    print("\n🐳 Docker 설정 파일들이 생성되었습니다:")
    print(f"  - {dockerfile_path}")
    print(f"  - {compose_path}")
    print(f"  - {deploy_script_path}")
    
    print("\n다음 명령어로 배포할 수 있습니다:")
    print(f"  chmod +x {deploy_script_path}")
    print(f"  ./{deploy_script_path}")
else:
    print("⚠️ Docker가 설치되어 있지 않습니다. 수동으로 파일을 생성합니다.")
    
    # Docker 파일들 수동 생성
    docker_manager.generate_dockerfile()
    docker_manager.generate_docker_compose()
    docker_manager.create_deployment_script()